# System preparation

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px
import sqlalchemy as db
import requests
import matplotlib.pyplot as plt
from sqlalchemy import text

# Data preparation

In [2]:
def POI_Station(kode_kota, conn, id_source = '6'):
    """Fungsi untuk mendapatkan stasiun pemadam kebakaran berdasarkan admin level kabupaten/kota
    Parameters
    ----------

    kode_kota : list of string
        kode kota dalam level kota/kabupaten
    conn : engine conn
        engine connection to database
    id_source : int
        id_source yang digunakan untuk data dari database
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    from sqlalchemy import text
    
    admin = Admin_Catch(kode_kota, conn)
    
    sql = text("""
    select id_merchant, nama_merchant, a.kode_brand, a.nama_brand, a.nama_sub_kategori,
    a.kode_sub_kategori,nama_kategori,kode_kategori, geom 
        from (select id_merchant, nama_merchant, a.kode_brand, a.nama_brand, a.nama_sub_kategori,a.kode_sub_kategori, geom, kode_kategori, nama_kategori
              from (select id_merchant, nama_merchant, B.kode_brand, B.nama_brand, C.kode_sub_kategori, C.nama_sub_kategori, geom, D.kode_kategori, D.nama_kategori
                    from (select id_merchant, id_desa,id_brand, nama_merchant, kode_desa, geom 
                          from tbl_merchant where status = 'T'
                          and ST_intersects('"""+str(admin.dissolve().geometry[0])+"""'::geometry, geom)) Z
                          left join tbl_brand B on B.id_brand = Z.id_brand
                          left join tbl_sub_kategori C on B.id_sub_kategori = C.id_sub_kategori 
                          left join tbl_kategori D on C.id_kategori = D.id_kategori 
                          and B.status_data = 'T' and C.status_data = 'T' and D.status_data = 'T')a)a
                          where left(kode_brand,4) =  '1204'
                          

    """)
    station = gpd.read_postgis(sql, conn, crs = 'epsg:4326')
        
    return station

In [3]:
def GRID(kode_kota, conn, id_source = '6', ukuran_grid = '1km'):
    """Fungsi untuk mendapatkan data geom grid menggunakan kode kota/kabupaten
    Parameters
    ----------
    kode_kota : list of string
        kode kota dalam level kota/kabupaten
    conn : engine conn
        engine connection to database
    id_source : int
        id_source yang digunakan untuk data dari database
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    from sqlalchemy import text
    if len(kode_kota)>1:
        sql = text("""
        select a.index as gid, grid_x, grid_y, grid_size, geom
        from tbl_grid_"""+ukuran_grid+""" a
        inner join (
                      select index 
                      from tbl_gid_to_kode_desa_n 
                      where id_source = """+id_source+""" and 
                      left(kode_desa,4) in """+str(tuple(kode_kota))+""") b on a.index = b.index""")
        grid = gpd.read_postgis(sql, conn, crs='epsg:4326')
        
    else:
        sql = text("""
        select a.index as gid, grid_x, grid_y, grid_size, geom
        from tbl_grid_"""+ukuran_grid+""" a
        inner join (
                      select index 
                      from tbl_gid_to_kode_desa_n 
                      where id_source = """+id_source+""" and 
                      left(kode_desa,4) = '"""+str(kode_kota[0])+"""') b on a.index = b.index""")
        grid = gpd.read_postgis(sql, conn, crs='epsg:4326')
    return grid

In [4]:
def Admin_Catch(kode_kota, conn, id_source='6'):
    """Fungsi untuk mendapatkan admin dalam level desa berdasarkan kode kota
    Parameters
    ----------

    kode_kota : list of string
        kode kota dalam level kota/kabupaten
    conn : engine connection
        engine connection to database
    id_source : int
        id_source yang digunakan untuk data dari database
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    from sqlalchemy import text
    
    if len(kode_kota)>1:
        sql = text("""select B.id_desa, B.nama_desa, A.id_kota,B.kode_desa, A.kode_kota,
        A.nama_kota, B.geom from tbl_master_desa_gabungan B
        join tbl_master_kota_gabungan A on left(B.kode_desa,4) = A.kode_kota
        where A.kode_kota in """+str(tuple(kode_kota))+"""  and 
        A.status_data = 'T' and
        B.status_data = 'T' and 
        B.id_source = {0} and 
        A.id_source = {0} """.format(id_source))

        admin = gpd.read_postgis(sql, conn, crs='epsg:4326')
    else:
        sql = text("""select B.id_desa, B.nama_desa, A.id_kota,B.kode_desa, A.kode_kota,
        A.nama_kota, B.geom from tbl_master_desa_gabungan B
        join tbl_master_kota_gabungan A on left(B.kode_desa,4) = A.kode_kota
        where A.kode_kota = '"""+str(kode_kota[0])+"""'  and 
        A.status_data = 'T' and
        B.status_data = 'T' and 
        B.id_source = {0} and 
        A.id_source = {0} """.format(id_source))

        admin = gpd.read_postgis(sql, conn, crs='epsg:4326')
    return admin

In [5]:
def Demog_Admin(kode_kota, conn, id_source='6', list_variable=['JUMLAH_PEN']):
    
    """Fungsi untuk mendapatkan admin dalam level desa yang tercatchment Buffer atau driving time
    Parameters
    ----------

    kode_kota : list of string
        kode kota dalam level kota/kabupaten
    conn : engine connection
        engine connection to database
    id_source : int
        id_source yang digunakan untuk data dari database
    list_variable : list
        list variable demografi yang akan digunakan sebagai analisis
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    import numpy as np
    from sqlalchemy import text
    
    
    if len(kode_kota)>1:
        sql = text("""select B.id_desa, B.nama_desa, A.id_kota,B.kode_desa, A.kode_kota,
        A.nama_kota, B.geom from tbl_master_desa_gabungan B
        join tbl_master_kota_gabungan A on left(B.kode_desa,4) = A.kode_kota
        where A.kode_kota in """+str(tuple(kode_kota))+"""  and 
        A.status_data = 'T' and
        B.status_data = 'T' and 
        B.id_source = {0} and 
        A.id_source = {0} """.format(id_source))

        admin = gpd.read_postgis(sql, conn, crs='epsg:4326')
    else:
        sql = text("""select B.id_desa, B.nama_desa, A.id_kota,B.kode_desa, A.kode_kota,
        A.nama_kota, B.geom from tbl_master_desa_gabungan B
        join tbl_master_kota_gabungan A on left(B.kode_desa,4) = A.kode_kota
        where A.kode_kota = '"""+str(kode_kota[0])+"""'  and 
        A.status_data = 'T' and
        B.status_data = 'T' and 
        B.id_source = {0} and 
        A.id_source = {0} """.format(id_source))

        admin = gpd.read_postgis(sql, conn, crs='epsg:4326')
    
    #select kode_desa
    kode_desa = admin.kode_desa.tolist()
    
    if len(kode_desa)>1:
        if len(list_variable)>1:
            #query to get data demografi
            sql = text("""
            select a.id, a.id_source, a.id_desa, a.variabel, b.deskripsi, 
            a.jumlah, a.id_kategori_thematic, a.kode_desa
            from tbl_jml_thematic_gabungan a 
            join tbl_kategori_thematic b on a.variabel = b.variabel
            join tbl_master_desa_gabungan c on c.kode_desa = a.kode_desa
            join tbl_master_kota_gabungan D on left(a.kode_desa,4) = D.kode_kota
            where b.variabel in """+str(tuple(list_variable))+""" and
            a.kode_desa in """+str(tuple(kode_desa))+""" and 
            a.id_source = """+id_source+""" 
            """)
            demog = pd.read_sql(sql, conn)
        else:
            #query to get data demografi
            sql = text("""
            select a.id, a.id_source, a.id_desa, a.variabel, b.deskripsi, 
            a.jumlah, a.id_kategori_thematic, a.kode_desa
            from tbl_jml_thematic_gabungan a 
            join tbl_kategori_thematic b on a.variabel = b.variabel
            join tbl_master_desa_gabungan c on c.kode_desa = a.kode_desa
            join tbl_master_kota_gabungan D on left(a.kode_desa,4) = D.kode_kota
            where b.variabel = '"""+list_variable[0]+"""' and
            a.kode_desa in """+str(tuple(kode_desa))+""" and 
            a.id_source = """+id_source+""" 
            """)
            demog = pd.read_sql(sql, conn)
    else :
        if len(list_variable)>1:
            #query to get data demografi
            sql = text("""
            select a.id, a.id_source, a.id_desa, a.variabel, b.deskripsi, 
            a.jumlah, a.id_kategori_thematic, a.kode_desa
            from tbl_jml_thematic_gabungan a 
            join tbl_kategori_thematic b on a.variabel = b.variabel
            join tbl_master_desa_gabungan c on c.kode_desa = a.kode_desa
            join tbl_master_kota_gabungan D on left(a.kode_desa,4) = D.kode_kota
            where b.variabel in """+str(tuple(list_variable))+""" and
            a.kode_desa = '"""+str(kode_desa[0])+"""' and 
            a.id_source = """+id_source+""" 
            """)
            demog = pd.read_sql(sql, conn)
        else:
            #query to get data demografi
            sql = text("""
            select a.id, a.id_source, a.id_desa, a.variabel, b.deskripsi, 
            a.jumlah, a.id_kategori_thematic, a.kode_desa
            from tbl_jml_thematic_gabungan a 
            join tbl_kategori_thematic b on a.variabel = b.variabel
            join tbl_master_desa_gabungan c on c.kode_desa = a.kode_desa
            join tbl_master_kota_gabungan D on left(a.kode_desa,4) = D.kode_kota
            where b.variabel = '"""+list_variable+"""' and
            a.kode_desa = '"""+str(kode_desa[0])+"""' and 
            a.id_source = """+id_source+""" 
            """)
            demog = pd.read_sql(sql, conn)
            
    #drop duplicates dari hasil query
    demog = demog.drop_duplicates()
    
    #Unmelted demographic table
    demog_unmelted = demog.pivot(index='kode_desa', columns='variabel', values='jumlah')
    demog_unmelted = demog_unmelted.reset_index()
    demog_unmelted.columns.name = None
    
    #join with gdf admin
    demog_join = pd.merge(admin,demog_unmelted,on='kode_desa',how='left')
    gdf_demog  = demog_join.replace(np.NaN, 0)
    
    return demog_join

In [6]:
def Demog_GRID(grid, demog_admin,list_variable=['JUMLAH_PEN'], Proj = 'epsg:6933'):
    """Fungsi untuk menghitung populasi pada setiap grid
    ----------

    grid = geopandas.GeoDataFrame()
        geodataframe grid
    demog_admin = deopandas.GeoDataFrame()
        Data demog dengan komponen populasi
    list_variable : list
        list variable demografi yang akan digunakan sebagai analisis
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    import numpy as np
    
    #Calculate percentage of SES
    percentage = pd.DataFrame()
    list_percent = []
    for i in grid.gid:
        gid = grid[grid['gid']==i].reset_index()
        x = gpd.clip(demog_admin, gid).reset_index()
        percentage['kode_desa'] = x.kode_desa.unique()
        percentage['percent'] = x.to_crs(Proj).area/gid.to_crs(Proj).area[0]
        percentage['gid'] = i
        list_percent.append(percentage)
        percentage = pd.DataFrame()
        percentage['gid'] = ""
        percentage['kode_desa'] = ""
        percentage['percent'] = ""
    percent = pd.concat(list_percent)
    
    columns = demog_admin.drop(columns = ['id_desa', 'nama_desa', 
                          'id_kota', 'kode_desa', 
                          'kode_kota',
                          'nama_kota', 'geom']).columns.tolist()
    variable = list_variable
    var = []
    not_var = []
    for i in variable:
        if i in columns:
            var.append(i)
        else:
            not_var.append(i)
    merge = pd.merge(percent, demog_admin[['kode_desa']+var])
    
    grid_cal = merge[['gid']]
    for i in var:
        grid_cal[i] = round(merge['percent']*merge[i])
    
    calculate = grid_cal.groupby('gid')[var].sum().reset_index()
    grid_calculate = pd.merge(grid, grid_cal)
    
    return grid_calculate[['gid','geom']+var]

In [7]:
def matrix_grid(grid, poi):
    """Fungsi untuk menghitung poi ke setiap grid
    ----------
    grid = geopandas.GeoDataFrame()
        geodataframe grid
    poi = deopandas.GeoDataFrame()
        Data POI Station
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    import numpy as np
    import json
    
    df = grid
    centroid = df.centroid
    centroid = gpd.GeoDataFrame(centroid.geometry)
    centroid['geometry'] = centroid[0]
    centroid.rename(columns = {'geometry':'geom'}, inplace =True)
    
    df = grid
    centroid = df.centroid
    centroid = gpd.GeoDataFrame(centroid.geometry)
    centroid['geometry'] = centroid[0]
    centroid.rename(columns = {'geometry':'geom'}, inplace =True)
    
    for j in poi.id_merchant:
        x = centroid
        y = poi[poi['id_merchant']==j]['geom']
        list_buff_dis = []
        for i in x.index:
            z = x[x.index==i].reset_index()['geom']
            merge = pd.DataFrame(pd.concat([y,z]))
            merge[['latitude', 'longitude']] = merge.apply(
                            lambda p: (p.geom.y, p.geom.x), axis=1, result_type='expand')
            coords = list(map(list, zip(merge['longitude'], merge['latitude'])))
            headers = {'Content-type': 'application/json', 'Accept': 'application/json'}
            profile = "truck"
            try:
                res = requests.post(os.getenv('API_MATRIX'), data=json.dumps(dict(profile=profile,points=coords)), headers=headers).json()
                distance = pd.DataFrame(res['time'][0])[1:]
                list_buff_dis.append(distance)
            except:
                distance = pd.DataFrame(data={0: [None]})
                list_buff_dis.append(distance)
            dis = pd.concat(list_buff_dis)
        df['t_'+str(j)] = dis[0].tolist()
    
    #Asumsi kcecepatan 40km/h atau 15 menit dalam 7km menurut UU NO 6 TH 2010
    process = pd.DataFrame()
    process2 = pd.DataFrame()
    for i in poi.id_merchant:
        process2[str(i)] = pd.DataFrame(df['t_'+str(i)]/60)
    df['Station'] = process2.idxmin(axis=1)
    df['time(minutes)'] = round(process2.min(axis=1))

        
    return df

In [8]:
def processing(demog_grid, matrix_grid):
    """Fungsi untuk mendapatkan klasifikasi risk
    ----------
    demog_grid = geopandas.GeoDataFrame()
        grid with demog data. Get from func Demog_GRID
    matrix_grid = geopandas.GeoDataFrame()
        demog with time or distance column. Get from func matrix_grid
    ----------
    """
    import pandas as pd
    import numpy as np
    
    var = demog_grid.drop(columns = ['gid','geom']).columns.tolist()
    if len(var) > 1:
        demog_grid['population'] = demog_grid[var].sum(axis=1)
    else :
        demog_grid['population'] = demog_grid[var]
    merge = pd.merge(demog_grid[['gid','population']], matrix_grid)
    merge.drop_duplicates('gid', keep='first', inplace=True)
    a = merge['population'].quantile()
    b = 15
    merge['class'] = np.where((merge['time(minutes)']>=b)&(merge.population<a), 1,
                              np.where(((merge['time(minutes)']<b)&(merge.population<a)), 2,
                                       np.where(((merge['time(minutes)']<b)&(merge.population>=a)), 3, 4)))
    merge['classification_risk'] = np.where(merge['class']==1,'Low Population Low Access',
                                        np.where(merge['class']==2,'Low Population High Access',
                                                 np.where(merge['class']==3,'High Population High Access',
                                                          'High Population Low Access')))
    
    return merge.set_geometry('geom')

In [137]:
def Quandrant_Vis(grid_cal):
    """Fungsi untuk mendapatkan visualisasi klasifikasi risk dalam kuandran 4 dimensi
    ----------
    grid_cal = geopandas.GeoDataFrame()
        grid with classification, demog, and matrix of time data
    ----------
    """
    import pandas as pd
    import matplotlib.pyplot as plt
    
    df = grid_cal.copy()
    x=np.array(df['time(minutes)'])
    y=np.array(df.population)
    xtick_labels=['High_Accessibility', 'Low_Accessibility']
    ytick_labels=['Low_Population', 'High_Population']
    ax=None

    # make the data easier to work with by putting it in a dataframe
    data = pd.DataFrame({'x': x, 'y': y})

    # let the user specify their own axes
    ax = ax if ax else plt.axes()

        # calculate averages up front to avoid repeated calculations
    y_avg = data['y'].quantile()
    x_avg = 15

        # set x limits
    adj_x = max((data['x'].max() - x_avg), (x_avg - data['x'].min()))*1.1
    lb_x, ub_x = (x_avg - adj_x, x_avg + adj_x)
    ax.set_xlim(lb_x, ub_x)

        # set y limits
    adj_y = max((data['y'].max() - y_avg), (y_avg - data['y'].min()))*1.1
    lb_y, ub_y = (y_avg - adj_y, y_avg + adj_y)
    ax.set_ylim(lb_y, ub_y)

        # set x tick labels
    if xtick_labels:
        ax.set_xticks([(x_avg - adj_x / 2), (x_avg + adj_x / 2)])
        ax.set_xticklabels(xtick_labels)

        # set y tick labels
    if ytick_labels:
        ax.set_yticks([(y_avg - adj_y / 2), (y_avg + adj_y / 2)])
        ax.set_yticklabels(ytick_labels, rotation='vertical', va='center')

        # plot points and quadrant lines
    ax.scatter(x=data['x'], y=data['y'], c='lightblue', edgecolor='darkblue',
    zorder=99)
    plt.title("Accesibility and Population")
    ax.axvline(x_avg, c='k', lw=1)
    ax.axhline(y_avg, c='k', lw=1)

def hex_to_RGB(hex_str):
    """ #FFFFFF -> [255,255,255]"""
    #Pass 16 to the integer function for change of base
    return [int(hex_str[i:i+2], 16) for i in range(1,6,2)]

def get_color_gradient(c1, c2, n):
    """
    Given two hex colors, returns a color gradient
    with n colors.
    """
    assert n > 1
    c1_rgb = np.array(hex_to_RGB(c1))/255
    c2_rgb = np.array(hex_to_RGB(c2))/255
    mix_pcts = [x/(n-1) for x in range(n)]
    rgb_colors = [((1-mix)*c1_rgb + (mix*c2_rgb)) for mix in mix_pcts]
    return ["#" + "".join([format(int(round(val*255)), "02x") for val in item]) for item in rgb_colors]

def Visualization(grid_cal, poi, col_name):
    import matplotlib.colors as colors
    """Fungsi untuk mendapatkan visualisasi based on column name
    ----------
    grid_cal = geopandas.GeoDataFrame()
        grid with classification, demog, and matrix of time data
    poi = geopandas.GeoDataFrame()
        data poi
    col_name = string
        nama kolom yang digunakan sebagai kolom klasifikasi (classification_risk, population, time(minutes))
    ----------
    """
    import pandas as pd
    import matplotlib.pyplot as plt
    import folium
    import geopandas
    import plotly.express as px
    grid_cal = grid_cal.reset_index().drop(columns = 'index')
    
    pallete = {'High Population Low Access':'#a62a21',
               'Low Population Low Access':'#faca50',
           'High Population High Access':'#f2d894',
           'Low Population High Access':'#f2ead5'
          }
    var = []
    not_var = []
    for i in ['High Population Low Access','Low Population Low Access','High Population High Access','Low Population High Access']:
        if i in grid_cal.classification_risk.unique():
            var.append(i)
        else :
            not_var.append(i)
    pal_ = {key: pallete[key] for key in var}
    color = [*pal_.values()]
    
    if col_name == 'classification_risk':
        m = grid_cal.explore(tiles = 'Cartodb Positron',
                             column = col_name,
                             cmap = colors.ListedColormap(color),
                             categories=var,
                             tooltip = ['time(minutes)','population','classification_risk']
                            )
        geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in poi.geometry]
        i = 0
        for coordinates in geo_df_list:
            folium.Marker(
            location=coordinates, 
            icon=folium.Icon(color="black",icon="fa-bus", prefix='fa'),
            popup=
                  "GID : " + str(grid_cal['gid'][i]) + "<br>"
                + "Station: " + str(poi['nama_merchant'][i]),
            ).add_to(m)
            i = i+1
        return m
    
    elif col_name == 'population':
        grid_cal['population'] = grid_cal['population'].astype(int)
        m = grid_cal.explore(tiles = 'Cartodb Positron',
                             column = col_name,
                             cmap = 'OrRd',
                            popup_kwds = dict(column = ''),
                            tooltip = [col_name],
                            popup = False,
                             k=5,
                             scheme = 'naturalbreaks',
                             legend_kwds=dict(colorbar=False, fmt='{:.0f}'),
                             legend = True,
                            )
        geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in poi.geometry]
        i = 0
        for coordinates in geo_df_list:
            folium.Marker(
            location=coordinates, 
            icon=folium.Icon(color="black",icon="fa-bus", prefix='fa'),
            popup=
                  "GID : " + str(grid_cal['gid'][i]) + "<br>"
                + "Station: " + str(poi['nama_merchant'][i]),
            ).add_to(m)
            i = i+1
        return m
    elif col_name == 'time(minutes)':
        grid_cal[col_name] = grid_cal[col_name].astype(int)
        m = grid_cal.explore(tiles = 'Cartodb Positron',
                             column = col_name,
                             cmap = 'OrRd_r',
                            
                            tooltip = [col_name],
                             k=5,
                             scheme = 'naturalbreaks',
                             popup_kwds = dict(column = ''),
                            popup = False,
                            legend_kwds=dict(colorbar=True, fmt='{:.0f}'),
                             legend = True,
                            )
        geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in poi.geometry]
        i = 0
        for coordinates in geo_df_list:
            folium.Marker(
            location=coordinates, 
            icon=folium.Icon(color="black",icon="fa-bus", prefix='fa'),
            popup=
                  "Station : " + str(poi['nama_merchant'][i]) + "<br>"
                ,
            ).add_to(m)
            i = i+1
        return m
    else:
        print('Wrong column Input')

def Position_Vis(longitude, latitude, grid_cal, poi):
    """Fungsi untuk mendapatkan visualisasi based on column name
    ----------
    grid_cal = geopandas.GeoDataFrame()
        grid with classification, demog, and matrix of time data
    poi = geopandas.GeoDataFrame()
        data poi
    ----------
    """
    import pandas as pd
    import matplotlib.pyplot as plt
    import folium
    import geopandas
    import plotly.express as px
    
    grid_cal = grid_cal.reset_index().drop(columns = 'index')
    #create dataframe
    df = pd.DataFrame({'longitude':[longitude],
                       'latitude':[latitude]})
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 'epsg:4326')
  
    location = gpd.sjoin(grid_cal, gdf[['geometry']]).reset_index()
    poi['id_merchant'] = poi['id_merchant'].astype(str)
    station = poi[poi['id_merchant']==location.Station.tolist()[0]].reset_index()
    station[['latitude', 'longitude']] = station.apply(
            lambda p: (p.geom.y, p.geom.x), axis=1, result_type='expand')
    
    m = grid_cal.explore(tiles = 'Cartodb Positron',
                             column = 'time(minutes)',
                             cmap = 'OrRd_r',
                            
                            tooltip = ['time(minutes)','population'],
                             k=5,
                             scheme = 'naturalbreaks',
                             popup_kwds = dict(column = ''),
                            popup = False,
                            legend_kwds=dict(colorbar=True, fmt='{:.0f}'),
                             legend = True,
                            )
    geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in poi.geometry]
    folium.Marker(
        location=[latitude, longitude],
        icon=folium.Icon(color="red",icon="fa-fire", prefix='fa'),
        popup=
              "Location : " + str(location['gid'][0]) + "<br>"
            + "Population: " + str(location['population'][0]) + "<br>"
            + "Time: " + str(grid_cal['time(minutes)'][0]) + "<br>"
            + "Risk Classification: " + str(grid_cal['classification_risk'][0])
            ,
    ).add_to(m)
    i = 0
    for coordinates in geo_df_list:
        folium.Marker(
        location=coordinates, 
        icon=folium.Icon(color="black",icon="fa-bus", prefix='fa'),
        popup=
              "GID : " + str(grid_cal['gid'][i]) + "<br>"
            + "Population: " + str(grid_cal['population'][i]) + "<br>"
            ,
        ).add_to(m)
        i = i+1
    gdf.rename(columns = {'geometry':'geom'}, inplace=True)
    merge = pd.concat([station[['geom']], gdf[['geom']]])
    loc = [[point.xy[1][0], point.xy[0][0]] for point in merge.geom]
    folium.PolyLine(loc,
                color='white',
                dash_array='10').add_to(m)
    
    
    return m

def Chart(grid_cal):
    """Fungsi untuk mendapatkan Chart Total population based on Time classification
    ----------
    grid_cal = geopandas.GeoDataFrame()
        grid with classification, demog, and matrix of time data
    ----------
    """
    import pandas as pd
    import numpy as np
    import geopandas
    import plotly.express as px
    color1 = '#9c2811'
    color2 = '#f2ead5'
    color = get_color_gradient(color1, color2, 9)
    grid_cal['time_class'] = np.where(grid_cal['time(minutes)'] < 5,'0 - 4 minutes',
        np.where(grid_cal['time(minutes)'] < 10, '5 - 9 minutes',
                np.where(grid_cal['time(minutes)'] < 15, '10 - 14 minutes',
                        np.where(grid_cal['time(minutes)'] < 20, '15 - 19 minutes',
                                np.where(grid_cal['time(minutes)'] < 25, '20 - 24 minutes',
                                        np.where(grid_cal['time(minutes)'] < 30, '25 - 29 minutes',
                                                np.where(grid_cal['time(minutes)'] < 35, '30 - 34 minutes',
                                                        np.where(grid_cal['time(minutes)'] < 40, '35 - 39 minutes','> 40 minutes'))))))))
    group = grid_cal.groupby('time_class')['population'].sum().reset_index()
    group.columns = ['Time','Total Population']
    list_sort = ['0 - 4 minutes',
                                            '5 - 9 minutes',
                                            '10 - 14 minutes',
                                            '15 - 19 minutes',
                                            '20 - 24 minutes',
                                            '25 - 29 minutes',
                                            '30 - 34 minutes',
                                            '35 - 39 minutes',
                                            '> 40 minutes']
    list_group = []
    for i in list_sort:
        test = group[group['Time']==i]
        list_group.append(test)
    group = pd.concat(list_group)
    
    group_test = pd.DataFrame({'Time':list_sort})
    group_merge = pd.merge(group_test, group, how='left')
    fig = px.bar(group_merge, x='Time', 
                 y='Total Population', 
                 color='Time',
                 color_discrete_map = {
                     '0 - 4 minutes':color[0],
                    '5 - 9 minutes':color[1],
                    '10 - 14 minutes':color[2],
                    '15 - 19 minutes':color[3],
                    '20 - 24 minutes':color[4],
                    '25 - 29 minutes':color[5],
                    '30 - 34 minutes':color[6],
                    '35 - 39 minutes':color[7],
                    '> 40 minutes' :color[8]
                 },
                 labels={'0 - 4 minutes',
                                            '5 - 9 minutes',
                                            '10 - 14 minutes',
                                            '15 - 19 minutes',
                                            '20 - 24 minutes',
                                            '25 - 29 minutes',
                                            '30 - 34 minutes',
                                            '35 - 39 minutes',
                                            '> 40 minutes'
            },
                 title = 'Total Population based on Time')
    fig.update_xaxes(categoryorder='array', 
                             categoryarray= list_sort)

    fig.show()